In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive' ,force_remount=True)

Mounted at /content/drive


In [ ]:
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py

rm: cannot remove 'seq2seq_trainer.py': No such file or directory
--2020-12-01 06:31:03--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9937 (9.7K) [text/plain]
Saving to: ‘seq2seq_trainer.py’

seq2seq_trainer.py  100%[===================>]   9.70K  --.-KB/s    in 0s      

2020-12-01 06:31:03 (107 MB/s) - ‘seq2seq_trainer.py’ saved [9937/9937]



In [ ]:
%%capture
!pip install -U datasets

In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 54099, done.
remote: Total 54099 (delta 0), reused 0 (delta 0), pack-reused 54099
Receiving objects: 100% (54099/54099), 40.37 MiB | 26.62 MiB/s, done.
Resolving deltas: 100% (37800/37800), done.


In [ ]:
!pip install -U ./transformers

Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 12.4MB/s 
     |████████████████████████████████| 2.9MB 48.5MB/s 
  Created wheel for transformers: filename=transformers-4.1.0.dev0-cp36-none-any.whl size=1371680 sha256=99ef9792d33c57a49112388442d5388f5a970d51a77d3200d99d440da4ecf843
  Stored in directory: /tmp/pip-ephem-wheel-cache-t2qdp3y_/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=71748348f9592376989b9dff1882d1ccc8a2e653b200be168dc70439045fdbed
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# !pip list

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

import pandas as pd
import torch
import datasets
from datasets import Dataset, load_dataset, load_metric
from sklearn.model_selection import train_test_split
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel, BertTokenizerFast, TrainingArguments

In [ ]:
from seq2seq_trainer import Seq2SeqTrainer

# New section

In [ ]:
tokenisation_batch_size = 1024
encoder_max_length = 512
decoder_max_length = 128

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/bertextabs')

In [ ]:
df = pd.read_csv('preprocess_data_part3.csv')
df = df[['more_clean_text','New Tagline']]
df = df.dropna()
df = df.fillna("")
df= df.iloc[0:500]
def generate_id(s):
    return abs(hash(s)) % (10 ** 10)

df['id'] = df['more_clean_text'].apply(generate_id)
df = df.applymap(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 519
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   more_clean_text  500 non-null    object
 1   New Tagline      500 non-null    object
 2   id               500 non-null    object
dtypes: object(3)
memory usage: 15.6+ KB


In [ ]:
ls

 bert2bert/                       preprocess_data_part3.csv
 bertabs_img.png                  preprocess_data_part4.csv
 checkpoint-12/                   preprocess_v1.csv
 checkpoint-24/                   __pycache__/
 checkpoint-36/                   requirements.txt
 checkpoint-48/                   runs/
 configuration_bertabs.py         seq2seq_trainer.py
 data/                           'seq2seq_training_args (1).py'
 finetune_bertabstraction.ipynb   seq2seq_training_args.py
 ft_cnndm_export.pkl              tagline.csv
 modeling_bertabs.py              tagline_v2.csv
 models/                          taglinge_v_3k.csv
 preprocess_data_part1.csv


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

In [ ]:
!rm -d checkpoint-100


rm: cannot remove 'checkpoint-100': No such file or directory


In [ ]:
print(type(train_data))
print(type(val_data))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
train_data[1]

{'New Tagline': 'payment solutions provider',
 '__index_level_0__': 450,
 'id': '6937256465',
 'more_clean_text': '0208 102 8000 info@cr7services.about cr7 services get in touch vaughan chambers | 4 tonbridge road maidstone | kent | me16 8rp 0208 102 8000 info@cr7services.get in touch vaughan chambers | 4 tonbridge road maidstone | kent | me16 8rp 0208 102 8000 info@cr7services'}

In [ ]:
def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["more_clean_text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["New Tagline"], padding="max_length", truncation=True, max_length=decoder_max_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]
    
    return batch

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=tokenisation_batch_size, 
    remove_columns=["more_clean_text","New Tagline","id","__index_level_0__"]
)

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=tokenisation_batch_size, 
    remove_columns=["more_clean_text","New Tagline","id","__index_level_0__"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

train_data.save_to_disk("data/encoded_curation_corpus_large_train_data")
val_data.save_to_disk("data/encoded_curation_corpus_large_val_data")

In [ ]:
# bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")
bert2bert = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

In [ ]:
bert2bert.save_pretrained("bert2bert")


In [ ]:
bert2bert

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:

# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 142
bert2bert.config.min_length = 56
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4




In [ ]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [ ]:
!pip install rouge-score


In [ ]:
rouge = load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score


     |████████████████████████████████| 163kB 7.9MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 17.1MB 215kB/s 


In [ ]:
from seq2seq_trainer import Seq2SeqTrainer
!rm seq2seq_training_args.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_training_args.py
from seq2seq_training_args import Seq2SeqTrainingArguments


--2020-11-30 22:11:30--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_training_args.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2071 (2.0K) [text/plain]
Saving to: ‘seq2seq_training_args.py’

seq2seq_training_ar 100%[===================>]   2.02K  --.-KB/s    in 0.001s  

2020-11-30 22:11:30 (3.05 MB/s) - ‘seq2seq_training_args.py’ saved [2071/2071]



In [ ]:
# from transformers import EncoderDecoderModel

# bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    #evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=1000,
    save_steps=12,#1000
    eval_steps=8000,
    warmup_steps=2000,
    overwrite_output_dir=True,
    save_total_limit=3
    #fp16=True
)

trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:850: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Step,Training Loss


TrainOutput(global_step=150, training_loss=5.321151529947917)

In [ ]:
!ls 

 bert2bert			  preprocess_data_part3.csv
 bertabs_img.png		  preprocess_data_part4.csv
 checkpoint-120			  preprocess_v1.csv
 checkpoint-132			  __pycache__
 checkpoint-144			  requirements.txt
 configuration_bertabs.py	  runs
 data				  seq2seq_trainer.py
 finetune_bertabstraction.ipynb  'seq2seq_training_args (1).py'
 ft_cnndm_export.pkl		  seq2seq_training_args.py
 modeling_bertabs.py		  tagline.csv
 models				  tagline_v2.csv
 preprocess_data_part1.csv	  taglinge_v_3k.csv


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("./checkpoint-144")
model.to("cuda")

test_data = Dataset.from_pandas(test_df)
test_data = test_data.select(range(30))

def generate_summary(batch):
    inputs = tokenizer(batch["more_clean_text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    batch["pred"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

results = test_data.map(generate_summary, batched=True, batch_size=16, remove_columns=["more_clean_text"])

pred_str = results["pred"]
label_str = results["New Tagline"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid
print(rouge_output)


Score(precision=0.057409466348736476, recall=0.23231783069987508, fmeasure=0.08877909170246726)


In [ ]:
!ls "models"

In [ ]:
test_data[20]['more_clean_text']

In [ ]:
label_str[20]

'determined and proven provider of agile manufacturing solutions based on additive manufacturing.'

In [ ]:
pred_str[20]

'to deliver we have built a new class of printers, optimized for precision speed and repeatability, to deliver. to deliver the new class, we have created a class of printer - based printers that will be available in the u. s. and europe, and the uk.'

In [ ]:
!pip install pytorch-pretrained-bert
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu

In [ ]:
 from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel
 
# Initializing a BERT bert-base-uncased style configuration
config_encoder = BertConfig()
config_decoder = BertConfig()
config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# Initializing a Bert2Bert model from the bert-base-uncased style configurations

model = EncoderDecoderModel(config=config)

# Accessing the model configuration
config_encoder = model.config.encoder
config_decoder  = model.config.decoder
# set decoder config to causal lm
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True

# Saving the model, including its configuration
model.save_pretrained('my-model')

# loading model and config from pretrained folder
encoder_decoder_config = EncoderDecoderConfig.from_pretrained('my-model')
model = EncoderDecoderModel.from_pretrained('my-model', config=encoder_decoder_config)

In [ ]:

from transformers import BertTokenizer, EncoderDecoderModel
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

#model = EncoderDecoderModel.from_pretrained("./checkpoint-3000")


model.to("cuda")

test_data = Dataset.from_pandas(test_df)
test_data = test_data.select(range(64))

def generate_summary(batch):
    inputs = tokenizer(batch["more_clean_text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    batch["pred"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

results = test_data.map(generate_summary, batched=True, batch_size=16, remove_columns=["more_clean_text"])

pred_str = results["pred"]
label_str = results["summary"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)